**A ANÁLISE DE SENTIMENTOS DE MENSAGENS DO TWITTER ENVOLVENDO SERVIÇOS DE ENTREGA DE ALIMENTOS**

- Autor/pesquisador: João Pedro Rodrigues Alves;
- Instituição: Universidade Presbiteriana Mackenzie;
- Curso: Ciência da Computação (FCI);
- Orientador: Ivan Carlos Alcantara de Oliveira;
- Apoio: PIBIC Mackenzie;

Código fonte do modelo de aprendizagem de máquina para análise de sentimentos de clientes dos apps iFood, Rappi e Zé Delivery. O modelo foi alcançado a partir de um estudo aprofundado em um projeto de iniciação científica de mesmo título. Para utilização faça uploud da base de dado no diretório do notebook ou faça as alterações necessárias para alterar o modo de acessar os dados (atente-se ao nome do arquivo e aos nomes das colunas para evitar conflito de nomes com o código fornecido abaixo).

In [ ]:
import re
import os
import nltk
import string
import unicodedata
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter
from nltk.stem import RSLPStemmer
from nltk.tokenize import word_tokenize
from imblearn.over_sampling import SMOTE
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
# Pega atual diretório de trabalho deste notebook e armazena-o em wd
wd = os.getcwd()
print(wd)

# Lista os arquivos na pasta dados
if not os.path.exists(wd):
  os.makedirs(wd)
os.listdir(wd)

In [ ]:
# Abre o Workbook e mostra o nome das planilhas
file = 'base_completa.xlsx'
df_base = pd.ExcelFile(file)
print(df_base.sheet_names)

df_dados = df_base.parse('Sheet1')
df_dados.info()

In [ ]:
# Exclui as colunas User e Date, as quais não serão relevantes na análise
df_dados = df_dados.drop(columns=['User', 'Date'])
df_dados.tail()

In [ ]:
# Função de geral de limpeza
def limpar_texto(texto):
    # Remove as citações de perfis (começado com @)
    texto = re.sub(r'@[\w\d_]+', ' ', texto)

    # Remove as hastags (começado com #)
    texto = re.sub(r'#[\w\d_]+', '', texto)

    # Remove a string RT isolada
    texto = re.sub(r'\bRT\b', '', texto)

    # Remove links de sites
    texto = re.sub(r'http\S+', '', texto)

    # Converte todo o texto para minúsculo
    texto = texto.lower()

    # Remove pontuações, emojis e caracteres especiais
    texto = re.sub(r'[^\w\s]|_', '', texto, flags=re.UNICODE)

    # Remove acentos
    texto = unicodedata.normalize('NFKD', texto).encode('ASCII', 'ignore').decode('ASCII')

    # Remove números
    texto = re.sub(r'\d+', '', texto)

    # Remove espaços extras
    texto = re.sub(r'\s+', ' ', texto).strip()

    return texto

In [ ]:
# Aplica a função de limpeza à coluna 'Tweet'
df_dados['Clean Tweet'] = df_dados['Tweet'].apply(limpar_texto)
df_dados.tail()

In [ ]:
# Função de tokenização e stemização
def tk_stemmer_text(text):
    tokens = word_tokenize(text.lower(), language='portuguese')
    stemmer = RSLPStemmer()
    normalized_tokens = [stemmer.stem(token) for token in tokens]
    return ' '.join(normalized_tokens)

In [ ]:
# Download de pacotes necesários para aplicar a função
nltk.download('punkt')
nltk.download('rslp')

In [ ]:
# Aplica a função de tokenização e stemização
df_dados['Tweet Stemming'] = df_dados['Clean Tweet'].apply(tk_stemmer_text)
df_dados.tail()

In [ ]:
# Separando entrada e saída do modelo
X = df_dados['Tweet Stemming']
y = df_dados['Feeling']

# Criando a matriz de features usando TF-IDF
tfidf = TfidfVectorizer()
X_tfidf = tfidf.fit_transform(X)

# Imprimindo a distribuição original das classes
print('Distribuição original das classes:', Counter(y))

# Divide os dados em conjunto de treino e teste
X_train_smote, X_test, y_train_smote, y_test = train_test_split(X_tfidf, y, test_size=0.2, random_state=42)
print('Distribuição original das classes treino:', Counter(y_train_smote))
print('Distribuição original das classes teste:', Counter(y_test))

In [ ]:
# Aplicação de SMOTE
smote = SMOTE(random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train_smote, y_train_smote)

# Imprimindo a nova distribuição das classes após SMOTE
print('Nova distribuição das classes após SMOTE:', Counter(y_train_smote))

In [ ]:
# Cria o modelo de Naive Bayes
clf = MultinomialNB()

# Treina o modelo
clf.fit(X_train_smote, y_train_smote)

# Realiza a previsão
y_pred = clf.predict(X_test)

# Gera a matriz de confusão
cm = confusion_matrix(y_test, y_pred)

# Imprime o relatório de classificação
cr = classification_report(y_test, y_pred)
print(cm)
print(cr)